In [ ]:
import sys
from IPython.display import Image, display
import sympy
sys.path.append('..')   # path for local package
import restflow

# Example 1: Burgers-KPZ equation

The KPZ equation is given by:
\begin{equation*}
\partial_t\phi = \kappa \nabla^2\phi + \frac{\lambda}{2} |\nabla\phi|^2  + \eta
\end{equation*}
The quantity $\phi$ denotes the coarse-grained height field and describes the dynamics of the surface growth <sup>1</sup>. The first term describes the smoothing mechanism through diffusion while the third term is Gaussian white noise. The second term is the non-linearity of the equation and describes the lateral surface growth. If this was zero, the model would be linear and thus exactly solvable. For more details on surface growth models and the KPZ equation, the reader can see <sup>2</sup>.

By switching to Fourier space, complications arise from the non-linear terms. These couple the Fourier wavectors due to the powers of the field. If such a term depends on $n$ wavectors then it is called an $n$-vertex function, denoted as $\cal{v}_n(\bold{k_1}, \cdots, \bold{k_n}|\bold{q})$ where $\bold{q}$ are the fourier coordinates of $\bold{r}$ respectively. In our case, the only non-zero vertex is a $2$-vertex that comes from the second term and is equal to:
\begin{equation*}
\cal{v}_2(\bold{k_1}, \bold{k_2}) = - \frac{\lambda}{2} \bold{k_1} \cdot \bold{k_2}
\end{equation*}
With this notation, the evolution equation for the field in Fourier space has the following form:
\begin{equation*}
\phi(\bold{q}) = G_0(\bold{q})\eta(\bold{q})+G_0(\bold{q})\int_{\bold{k_1}}\int_{\bold{k_2}}\cal{v_2}(\bold{k_1}, \bold{k_2})\phi(\bold{k_1})\phi(\bold{k_2})(2\pi)^{d+1}\delta(\bold{q}-\bold{k_1}-\bold{k_2})
\end{equation*}
where $G_0(\bold{q})$ is the bare propagator obtained from the linear terms and is equal to $G_0(\bold{q})=\frac{1}{-i\omega+f(q)}$ with $f(k)=\kappa k^2$. 

To solve this integral equation, we plug the RHS recursively into $\phi(\bold{k_1})$ and $\phi(\bold{k_2})$ and we get a series of terms with increasing powers of the vertex function. Using the Wilson's shell renormalization, this becomes a perturbation series with respect to the shell momentum $\delta l$ (for more details see Section 3.4).

We represent every term from this series as a graph $\Gamma_n$ (see Section 3.2). These can be translated into nested integrals $\cal{I}(\Gamma_n; \bold{x})$ (see Section 3.3) which depend in principle on the outgoing wave vectors. By summing all the graphs with the same number $n$ of outgoing wavectors, we <i>graphically correct</i> the vertex functions and the propagators as:
\begin{equation*}
\cal{\tilde{v}}_n(\bold{p_1}, \cdots, \bold{p_n}) = \cal{v}_n(\bold{p_1}, \cdots, \bold{p_n}) + \sum_m \cal{I}(\Gamma_n^{(m)}; \bold{p_1}, \cdots, \bold{p_n})
\end{equation*}  
We call the second term with the sum as the correction to the vertices. We first correct the propagator and then the vertices. Its corrections are given by constructing all the possible graphs with one external leg using the non-zero vertex (See Figure 2 from manuscript).

We will only consider 1-loop graphs. For more loops, their contributions are of higher order in the perturbation series. 

Let's go back to the KPZ case. The only relevant one-loop graph correcting the propagator for this example is:

In [ ]:
pil_img = Image(filename='./figures/kpz_graph.jpg')
display(pil_img)

The goal of this program is to translate these graphs into nested integrals and then to solve these integrals in the spirit of Wilson's shell renormalization to obtain the corrections of the vertex functions. Below are the steps to achieve this applied to the KPZ model.

<b><ins>1st Step:</b></ins> Define the model parameters, the vectors, the propagators and the vertex functions:

In [ ]:
# define the model
class KPZ:
    def __init__(self):
        self.alpha = 0
        self.lam, self.kap, self.D = sympy.symbols('lambda kappa D')

    def f(self,k):
        return self.kap*k**2

    def v2(self,k1,k2,q):
        return -.5*self.lam*k1*k2,1

# symbols for vectors
_q, _k, dot_kq = sympy.symbols('q k (k·q)')
# saves the vectors and dot products in a class
ctx = restflow.Context()
ctx.add_dot_product(_q,_k,dot_kq)
# create symbolic wave vectors
k = ctx.vector(_k)
q = ctx.vector(_q)

model = KPZ()

<b><ins>2nd Step:</b></ins> Create the graph:

In [ ]:
v = [restflow.Vertex() for i in range(3)]
v[0].link_vertex(v[2], angle=0.12)
v[0].link_vertex(v[1], 0.0)
v[1].link_vertex(v[2], 0.0)
v[1].add_outgoing(0.0)
g = restflow.Graph(v)

<b><ins>3rd Step:</b></ins> Label the graph by giving an input `labels` and using `label_edges`. 
(Optional): Visualize it using `plot_graph`. Create a LaTeX file for nicer rendering using `export_latex_graph`:

In [ ]:
labels = [k, q] # input array of sink vector and outgoing legs
g.label_edges(labels)
g.plot_graph()
g.export_latex_graph('graph_plot')

<b><ins>4th Step:</b></ins> Convert graph into integral using ```convert```. Here we also change coordinates $\mathbf{k} \rightarrow \mathbf{k}+\mathbf{q}/2$ by changing the `labels` input:

In [ ]:
# change of coordinates by renaming the labels
labels = [.5*q+k, q] # input array of sink vector and outgoing legs
g.label_edges(labels)
expr = g.convert(model)
display(expr.num/expr.den)

<b><ins>5th Step:</b></ins> Calculate the integral using `integrate`:

In [ ]:
res = restflow.integrate([expr],3,labels)
display(res)

This integral corrects the model parameter $\kappa$. Performing the wavector integral and renormalizing it, we get:

\begin{equation*}
\psi_\kappa = \frac{D\lambda^2 K_d \Lambda^{d-2}}{\kappa^3} \frac{2-d}{4d}
\end{equation*}

For details how to get the graphical corrections from the integrals see example ```neural_network.ipynb```. 

# Example 2: Graph with 2 external legs

We graphically corrected the propagator. To correct the 2-vertex, we need to consider all the graphs with 2 external legs. There are 2 1-loop graphs with 2 external legs for only 2-vertices:

In [ ]:
pil_img = Image(filename='./figures/kpz_2vertex.jpg')
display(pil_img)

We start with graph (A):

In [ ]:
pil_img = Image(filename='./figures/kpz_2vertex_graphb.jpg')
display(pil_img)

This graph actually represents 2 further graphs (figure (A1) and (A2)) depending on the labeling of the external legs. 

To set up the problem: Use the same system with before but we need to define the vector $p$ and the corresponding dot products:

In [ ]:
_p, dot_pk, dot_qp = sympy.symbols('p (k·p) (q·p)')
ctx.add_dot_product(_p,_k,dot_pk)
ctx.add_dot_product(_q,_p,dot_qp)
# create symbolic wave vectors
p = ctx.vector(_p)

Create the graph:

In [ ]:
v = [restflow.Vertex() for i in range(4)]
v[0].link_vertex(v[1])
v[0].link_vertex(v[2])
v[2].link_vertex(v[3])
v[3].link_vertex(v[1])
v[2].add_outgoing()
v[3].add_outgoing()
g = restflow.Graph(v)
labels = [k, p, q-p]
g.label_edges(labels)


Use the method `convert_perm` to calculate all the permutations of the external momenta (figure (b) and (c)):

In [ ]:
exprs = g.convert_perm(model,labels)
for expr in exprs:
    display(expr.num/expr.den)

Use the function `integrate` to calculate the integrals and sum the two graphs:

In [ ]:
I_A = restflow.integrate(exprs,3,labels)
display(I_A)

We follow the same procedure for graph B:

In [ ]:
v = [restflow.Vertex() for i in range(4)]
v[0].link_vertex(v[1])
v[0].link_vertex(v[2])
v[1].link_vertex(v[3])
v[2].link_vertex(v[3])
v[1].add_outgoing()
v[2].add_outgoing()
g = restflow.Graph(v)
labels = [k, p, q-p]
exprs = g.convert_perm(model,labels)
I_B = restflow.integrate(exprs,3,labels)
display(I_B)

We add the graphical contributions to the 2-vertex from these graphs:

In [ ]:
I_2vertex = sympy.simplify(I_A+I_B)
display(I_2vertex)

The graphical correction for 1-loop graphs for the $\lambda$ parameter for the KPZ model is zero! This turns out to be true for higher order loops. The reason is an underlying symmetry from Galilean invariance which describes infinitesimal tilting of the interface <sup>3</sup>.

Once we have the corrections to the propagator and the 2-vertex, we can renormalize the model parameters by comparing the coefficients of the renormalized and unrenormalized vertex functions. Then, by proper scaling to restore the cut-off, the Wilson's flow equations can be found (see section 3.4). 

In the example in  ```neural_network.ipynb```, we illustrate how to calculate the final flow equations.



# References

<sup>1</sup> Kardar, Mehran, Giorgio Parisi, and Yi-Cheng Zhang. "Dynamic scaling of growing interfaces." Physical Review Letters 56.9 (1986): 889.

<sup>2</sup> Krug, Joachim. "Origins of scale invariance in growth processes." Advances in Physics 46.2 (1997): 139-282.

<sup>3</sup> Medina, Ernesto, et al. "Burgers equation with correlated noise: Renormalization-group analysis and applications to directed polymers and interface growth." Physical Review A 39.6 (1989): 3053.